<a href="https://colab.research.google.com/github/bella0826/MNIST/blob/main/MNIST_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import matplotlib.pyplot as plt

In [3]:
EPOCHS = 10
BATCH_SIZE = 32
LR = 0.001
DOWNLOAD = True

In [4]:
train_data = torchvision.datasets.MNIST(root = "./mnist", train = True, transform = torchvision.transforms.ToTensor(),download = DOWNLOAD)
test_data = torchvision.datasets.MNIST(root = "./mnist", train = False, transform = torchvision.transforms.ToTensor())

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist/MNIST/raw



In [5]:
train_loader = DataLoader(dataset = train_data, batch_size = BATCH_SIZE, shuffle = True, num_workers = 2)
test_loader = DataLoader(dataset = test_data, batch_size = BATCH_SIZE)

In [16]:
class Model(nn.Module):
  def __init__(self):
    super(Model,self).__init__()  
    #input_size = (1,28,28)
    self.conv1 = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    ) #output_size = (16,14,14)
    self.conv2 = nn.Sequential(
        nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    ) #output_size = (32,7,7)
    self.fc1 = nn.Linear(32*7*7,128)
    self.fc2 = nn.Linear(128,10)
  def forward(self,x):
    out = self.conv1(x)
    out = self.conv2(out)
    out = out.view(out.size(0),-1)
    out = self.fc1(out)
    out = self.fc2(out)
    out = F.softmax(out,dim=1)
    return out

In [7]:
def train(model,train_loader,optimizer,criterion,epochs):
  model.train()
  train_acc_his = []
  train_loss_his = []
  
  for epoch in range(1,epochs+1):
    train_size = 0
    for idx,(data,target) in enumerate(train_loader):
      optimizer.zero_grad()
      output = model(data)
      #pred = output.argmax(dim=1,keepdim=True)
      #train_correct += pred.eq(target.view_as(pred)).sum().item()
      loss = criterion(output,target)
      loss.backward()
      optimizer.step()
      #train_size += data.size(0)
      if idx % 50 == 0:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, idx * len(data), len(train_loader.dataset), 100. * idx / len(train_loader), loss.item()))


In [8]:
def test(model,test_loader,criterion):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for idx,(data,target) in enumerate(test_loader):
      output = model(data)
      test_loss += criterion(output,target).item()
      pred = output.argmax(dim=1,keepdim=True)
      correct += pred.eq(target.view_as(pred)).sum().item()
  test_loss /= len(test_loader.dataset)
  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))

In [17]:
cnn = Model()
optimizer = optim.Adam(cnn.parameters(),lr=LR)
criterion = nn.CrossEntropyLoss()
train(cnn,train_loader,optimizer,criterion,EPOCHS)
test(cnn,test_loader,criterion)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300370
Train Epoch: 1 [1600/60000 (3%)]	Loss: 1.895919
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.688289
Train Epoch: 1 [4800/60000 (8%)]	Loss: 1.633336
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.671937
Train Epoch: 1 [8000/60000 (13%)]	Loss: 1.602172
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.541004
Train Epoch: 1 [11200/60000 (19%)]	Loss: 1.493725
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.498800
Train Epoch: 1 [14400/60000 (24%)]	Loss: 1.510772
Train Epoch: 1 [16000/60000 (27%)]	Loss: 1.548147
Train Epoch: 1 [17600/60000 (29%)]	Loss: 1.521049
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.512742
Train Epoch: 1 [20800/60000 (35%)]	Loss: 1.502306
Train Epoch: 1 [22400/60000 (37%)]	Loss: 1.463075
Train Epoch: 1 [24000/60000 (40%)]	Loss: 1.462432
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.548196
Train Epoch: 1 [27200/60000 (45%)]	Loss: 1.493526
Train Epoch: 1 [28800/60000 (48%)]	Loss: 1.524937
Train Epoch: 1 [30400/60000 (51%)]	Loss: 1.496868
Train Epoch: 1